In [2]:
import os
import pdfplumber
import pandas as pd


In [ ]:
folder_path = r'C:\data_insights\developement\aug_plano'
destination_folder = r'C:\data_insights\developement\aug_plano'

excel_file = os.path.join(destination_folder, 'POGs Sci Q Stores Aug.xlsx')
writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')

for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
        pdf_file = os.path.join(folder_path, filename)

        # Extract month and day from PDF file name
        file_name = filename.split('.')[0]  # Remove the file extension
        day = file_name.split('_')[-2]  # Extract the day from the file name
        month = file_name.split('_')[-3].capitalize()  # Extract the month from the file name and capitalize it

        # Create the sheet name
        sheet_name = f'{day} POG {month}'

        # Open the PDF file
        with pdfplumber.open(pdf_file) as pdf:

            # Define the header
            header = ['BIN', 'GAME #', 'GAME NAME', 'PRICE']
            modified_header = header + ['FACINGS']
            recording = False  # Flag to start recording rows
            rows = []  # Store the rows

            # Loop through each page in the PDF
            for page in pdf.pages:

                # Extract the tables from the page
                tables = page.extract_tables()

                # Loop through each table
                for table in tables:

                    # Process each row in the table
                    for row in table:

                        # If header is found, start recording
                        if row[:4] == header:
                            recording = True

                        # If recording flag is True and row has enough elements, process the row
                        if recording and row[:4] != header and len(row) >= 3:
                            # Replace '\n' with ' ' in the "GAME NAME" column
                            row[2] = row[2].replace('\n', ' ')

                            # Add the "FACINGS" column
                            if row[0]:  # If the "BIN" field is not blank
                                row.append(1)
                            else:
                                row.append('')

                            rows.append(row)
            # Close sequence 
            df = pd.DataFrame(rows, columns=modified_header)
            df.to_excel(writer, sheet_name=sheet_name, index=False)
            pdf.close()
writer.save()  # Save the Excel writer
writer.close()